In [4]:
import telegram
from telegram.ext import Updater
from telegram.ext import CommandHandler
from telegram.ext import MessageHandler, Filters
import requests 
import json
import pandas as pd
from bs4 import BeautifulSoup

    
###     JSON file example    ###
#
# {
#   "1": [
#     {
#       "stock_number": "005930",
#       "stock_name": "삼성전자",
#       "stock_price": "57500"
#     }
#   ],
#   "2": [
#     {
#       "stock_number": "000660",
#       "stock_name": "LG전자",
#       "stock_price": "85600"
#     }
#   ]
# }
#
###     JSON file example    ###


# JSON data read
with open('./stock_data.json' ,encoding = "UTF-8") as file:
    json_data = json.load(file)
    temp = pd.DataFrame(json_data)
    df = temp.T
    
# Active Bot with the token
token = "1461066934:AAHQLhV3b7PJ-EJX-JNJdYv0tz2fZBnWmhs"
bot = telegram.Bot(token)

# updater 
updater = Updater(token=token, use_context=True)
dispatcher = updater.dispatcher

# Getting Stock Information
def get_stock_price(code):
    result = requests.get(url+code)
    bs_obj = BeautifulSoup(result.content,"html.parser")
    no_today = bs_obj.find("p", {"class": "no_today"})
    blind = no_today.find("span", {"class": "blind"}) # 태그 span, 속성값 blind 찾기
    now_price = blind.text
    return now_price

# Pre-requirement Data
url = "https://finance.naver.com/item/main.nhn?code=" 
ment1 = "<<  끝투's 오늘의 추천 종목  >>\n\n"
ment = ment1
image = 'test_image.jpg'

# make stock name as same length with each other
def equalizer(a):
    gong = ""
    for i in range (9-a):
        gong = gong + "  "
    return gong    

# make 'ment' for sending telegram
for idx,row in df.iterrows():
    info = row
    code = info['stock_number']
    code_name = info['stock_name']
    name_len_a = len(code_name)
    price = info['stock_price']
    price = str(price)
    price = price.rjust(8)
    url_with_code = url + code
    updated_url = "[" + code_name + "]" + "(" + url_with_code + ")"
    temp = idx + '. ' + updated_url + equalizer(name_len_a) +'가격'+price + '    매수 클릭\n\n'   ####you can use get_stock_price(code) for the price
    ment = ment + temp
        
# send message to users
bot.send_photo(chat_id = 1437875774, photo=open(image, 'rb'))
bot.sendMessage(chat_id = 1437875774,text = ment, parse_mode = 'Markdown',disable_web_page_preview=True)



KeyError: 'stock_number'